In [40]:
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import Callback,ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical 
import os
from sklearn.model_selection import train_test_split
import random 

In [42]:
image_height, image_width = 84, 84
batch_size = 180
learning_rate = 0.001
num_classes = 4
epochs = 30
train_image_path = '../input/pollen-grains/new/train'
test_image_path = '../input/pollen-grains/new/test'
train_labels =[1,2,3,4]

In [43]:
train_datagen = ImageDataGenerator(rescale = 1./255,validation_split = 0.2,
#         rotation_range=40,
#         height_shift_range=0.2,
#         width_shift_range=0.2,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest' 
                                  )

val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [44]:
print('Total number of images for training: ')
train_generator = train_datagen.flow_from_directory(train_image_path, 
                                                    class_mode = "categorical",
                                                    subset ='training',
                                                    shuffle =True,
                                                    target_size=(image_height, image_width),
                                                    batch_size=batch_size)                                                          

In [45]:
print('Total number of images for validation: ')
validation_generator = train_datagen.flow_from_directory(train_image_path, 
                                                    subset ='validation',
                                                    #validation_split = 0.2,                       
                                                    seed =1339,
                            
                                                    target_size=(image_height, image_width),
                                                    batch_size=batch_size,
                                                    class_mode = "categorical" )                       
                                                   

In [46]:
nb_train_samples = len(train_generator.filenames) 
num_classes = len(train_generator.class_indices)
print("Number of classes: ", num_classes)  

In [47]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense ,BatchNormalization
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers, models, Model, optimizers
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

model = Sequential()
model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)))
#model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(16, (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(layers.Dropout(0.6))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [49]:
checkpoint = ModelCheckpoint("fina_project_model.h5", 
                             monitor = 'val_acc',
                             verbose=1, 
                             save_best_only = True, 
                             save_Weights_only = False, 
                             mode = 'auto', save_freq=1)

In [50]:
model.compile(loss = "categorical_crossentropy", 
              optimizer = optimizers.Adam(learning_rate = learning_rate, clipnorm = 1.),
               metrics=['acc',f1_m,precision_m, recall_m])
#metrics = ['acc'],

In [51]:
history = model.fit(train_generator,
                    batch_size= batch_size,   #stepPerEpoch_training,
                    epochs=epochs,
                    shuffle=True,
                    validation_data=validation_generator,
                    callbacks=[checkpoint]
                   )

In [52]:
import matplotlib.pyplot as plt

#model = models.load_model("final_project_model.h5")

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.figure(1, figsize = (15,8))
plt.subplot(221)
plt.plot(epochs, acc, 'b', label = 'Training acc')
plt.plot(epochs, val_acc, 'r', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
#plt.savefig('Accuracy.jpg')

plt.subplot(222)
plt.plot(epochs, loss, 'b', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()